In [ ]:
step_loss <- function(x, y, c1) {
loss <- var(y[x < c1]) + var(y[x >= c1])
return(loss)
}
step_minimizer <- function(x, y) {
c1 <- optim(c(c1 = median(x)), fn = step_loss, x = x, y = y,
method = "Brent", lower = min(x), upper = max(x))
return(c1$par)
}
step_comparitor <- function(x, y, c1, level) {
# Compare the p-value from a t-test to the `level` argument
do_split <- t.test(y[x < c1], y[x >= c1])$p.value <= level
return(do_split)
}
find_knots <- function(x, y, level = 0.05, minsize = 10) {
# Recursively traverse either side of each knot,
# recording all knots as it goes.
if (length(x) < minsize) {
return(numeric(0)) # Don't split (stop)
}
c1 <- step_minimizer(x, y)
do_first_split <- step_comparitor(x, y, c1, level)
if (do_first_split) {
left_split <- find_knots(x[x < c1], y[x < c1],
level = level)
right_split <- find_knots(x[x >= c1], y[x >= c1],
level = level)
splits <- sort(c(c1, left_split, right_split))
return(splits)
} else {
return(numeric(0)) # Don't split (stop)
}
}

x <- runif(n = 100, 0, 10)
knots <- c(2, 6)
g <- cut(x, c(-Inf, knots, Inf), labels = FALSE)
y <- rnorm(length(x), mean = c(0, 4, 7)[g], sd = 1)
estimated_knots <- find_knots(x, y, level = 0.05)
print(estimated_knots)
plot(x, y)
abline(v = estimated_knots)